# This notebook is exclusively to create the training data set

In [40]:
# Modues to be used
import pickle
import json
import numpy as np
import pandas as pd

from datetime import datetime

In [2]:
# Load the original data

# -> Player dictionary with the dataFrames
f = open('../data/dict_player.pickle', 'rb')
dict_player = pickle.load(f)

# -> Map dataFrames
f = open('../data/df_map.pickle','rb')
df_map = pickle.load(f)

# -> Map dictionary
f = open('../data/dict_map.pickle','rb')
dict_map = pickle.load(f)

In [50]:
# Loop throughout all of the players and put it in the right map
df_map_ranked = ( df_map[ (df_map['date']>datetime(2016,6,6)) & (df_map['team_rank_1']>0) &  (df_map['team_rank_2']>0) ] )


36625

In [108]:

metric_vec = ['KAST', 'ADR', 'first_kills_diff', 'rating',
       'kills_per_round', 'deaths_per_round', 'impact', 
       'team_score',
       'op_score', 'win', 'team_rank', 'prize']

map_training_dict = {}

for map_id in df_map_ranked['map_id'][:]:
    
    map_training_dict[map_id] = {}

    # Keys for the dictionary
    map_training_dict[map_id]['t1_win']    = 0
    map_training_dict[map_id]['fav_win']   = 0
    map_training_dict[map_id]['map']       = df_map_ranked.at[map_id,'map']
    map_training_dict[map_id]['score_dif'] = 0
    map_training_dict[map_id]['t1_rank']   = 0
    map_training_dict[map_id]['t2_rank']   = 0

    # Get who started CT
    for t in range(0,2):
        for p in range(0,5):
            for metric in metric_vec:
                map_training_dict[map_id][
                    't_'+str(t)+'_p_'+str(p)+'_'+metric] = {}

    with open('/home/emmanuel/Desktop/csgo-csv/json_maps/hltv_map_'+str(map_id)+'.json') as f:

        data = json.load(f)
        id_ct_start = data['roundHistory'][0]['ctTeam']

    team_1  = df_map.at[map_id,'team_id_1']
    rank_1  = df_map.at[map_id,'team_rank_1']
    score_1 = df_map.at[map_id,'team_score_1']

    team_2  = df_map.at[map_id,'team_id_2']
    rank_2  = df_map.at[map_id,'team_rank_2']
    score_2 = df_map.at[map_id,'team_score_2']

    if team_1 == id_ct_start:
        map_training_dict[map_id]['score_dif'] = score_1-score_2
        map_training_dict[map_id]['t1_rank']   = rank_1
        map_training_dict[map_id]['t2_rank']   = rank_2

        if score_1 > score_2:
            map_training_dict[map_id]['t1_win'] = 1
        else:
            map_training_dict[map_id]['t1_win'] = 0

    else:
        map_training_dict[map_id]['score_dif'] = score_2-score_1
        map_training_dict[map_id]['t1_rank']   = rank_2
        map_training_dict[map_id]['t2_rank']   = rank_1

        if score_2 > score_1:
            map_training_dict[map_id]['t1_win'] = 1
        else:
            map_training_dict[map_id]['t1_win'] = 0

    if ( (map_training_dict[map_id]['score_dif'] > 0) & (map_training_dict[map_id]['t1_rank']<map_training_dict[map_id]['t2_rank']) )|( (map_training_dict[map_id]['score_dif'] < 0) & (map_training_dict[map_id]['t2_rank']<map_training_dict[map_id]['t1_rank']) ):
        map_training_dict[map_id]['fav_win'] = 1
    

In [84]:
map_training_dict
#df_map.keys()

_prize': {},
  't_1_p_4_KAST': {},
  't_1_p_4_ADR': {},
  't_1_p_4_first_kills_diff': {},
  't_1_p_4_rating': {},
  't_1_p_4_kills_per_round': {},
  't_1_p_4_deaths_per_round': {},
  't_1_p_4_impact': {},
  't_1_p_4_team_score': {},
  't_1_p_4_op_score': {},
  't_1_p_4_win': {},
  't_1_p_4_team_rank': {},
  't_1_p_4_prize': {}},
 31510: {'t1_win': 0,
  'fav_win': 0,
  'map': 'cch',
  'score_dif': -8,
  't1_rank': 136,
  't2_rank': 36,
  't_0_p_0_KAST': {},
  't_0_p_0_ADR': {},
  't_0_p_0_first_kills_diff': {},
  't_0_p_0_rating': {},
  't_0_p_0_kills_per_round': {},
  't_0_p_0_deaths_per_round': {},
  't_0_p_0_impact': {},
  't_0_p_0_team_score': {},
  't_0_p_0_op_score': {},
  't_0_p_0_win': {},
  't_0_p_0_team_rank': {},
  't_0_p_0_prize': {},
  't_0_p_1_KAST': {},
  't_0_p_1_ADR': {},
  't_0_p_1_first_kills_diff': {},
  't_0_p_1_rating': {},
  't_0_p_1_kills_per_round': {},
  't_0_p_1_deaths_per_round': {},
  't_0_p_1_impact': {},
  't_0_p_1_team_score': {},
  't_0_p_1_op_score': {}

In [109]:
df_ct_start = pd.DataFrame.from_dict( map_training_dict,orient='index')
len( df_ct_start )

# type(data['roundHistory'][0]['ctTeam'])
#df_map.loc[34200]

36625

In [120]:
df_ct_start.tail(n=20)
print(df_ct_start[df_ct_start['map']=='inf']['t1_win'].mean())
df_ct_start['t1_rank'].max()
# print(df_ct_start['fav_win'].mean())

0.5176948051948052


395

In [151]:

count = 0

#del df

for player_id in dict_player:
    
    if count == 0:
        df = dict_player[player_id].copy().reset_index()
    else:
        print(1)
        print(dict_player[player_id]['player_name'].unique())
        df = df.append(dict_player[player_id].reset_index(), ignore_index=True )
    
    if count > 5:
        break

    count = count+1

1
['Helsing']
1
['Xizt']
1
['f0rest']
1
['RpK']
1
['fox']
1
['TorPe']


In [153]:
len(df.sort_values(['map_id']))

6925

In [69]:
dict_map

#dict_player[30].keys()

14107: {4411: {'score': 16,
   'players_id': array([  39,   29,  884, 7148, 7147]),
   'win': 1},
  4555: {'score': 10,
   'players_id': array([334, 339, 737, 735, 302]),
   'win': 0}},
 14108: {4759: {'score': 6,
   'players_id': array([ 373, 7181,  353,  349, 4382]),
   'win': 0},
  4472: {'score': 16,
   'players_id': array([7387, 7188, 7186, 7184,  414]),
   'win': 1}},
 14109: {4759: {'score': 8,
   'players_id': array([ 373,  353, 7181, 4382,  349]),
   'win': 0},
  4472: {'score': 16,
   'players_id': array([7387, 7188, 7186,  414, 7184]),
   'win': 1}},
 14114: {4501: {'score': 15,
   'players_id': array([7323, 7216, 7218, 7219, 7217]),
   'win': 0},
  4615: {'score': 15,
   'players_id': array([1225,  146, 7581, 7391,  150]),
   'win': 0}},
 14115: {4494: {'score': 16,
   'players_id': array([429, 128, 102,  92,  77]),
   'win': 1},
  4685: {'score': 9,
   'players_id': array([3159, 7491, 7404, 7489, 2154]),
   'win': 0}},
 14116: {4494: {'score': 16,
   'players_id': array([1